<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/CaptumLongformerSequenceClassificationMultiembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 55.1 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 71.3 MB/s 


In [ ]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 4.1 MB/s 


In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 57.0 MB/s 
     |████████████████████████████████| 140 kB 54.6 MB/s 
     |████████████████████████████████| 212 kB 49.7 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 144 kB 54.7 MB/s 
     |████████████████████████████████| 271 kB 22.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = '/content/drive/MyDrive/cogs402longformer/models/longformer-finetuned_papers/checkpoint-2356'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
# model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased")

In [ ]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='allenai/longformer-base-4096', vocab_size=50265, model_max_len=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})


In [ ]:
def predict(inputs, position_ids=None):
    output = model(inputs,
                   position_ids=position_ids)
    return output.logits

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
max_length = 2048
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = 2048)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [ ]:
from datasets import load_dataset
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402dataset-cc784554b797f843


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-cc784554b797f843/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
testval = 923
text = cogs402_ds['text'][testval]
label = cogs402_ds['labels'][testval]
print(label)

0


In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   )
    return torch.softmax(preds, dim = 1)

In [ ]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [ ]:
print(position_ids)
print(ref_position_ids)

tensor([[   0,    1,    2,  ..., 2047, 2048, 2049]], device='cuda:0')
tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')


In [ ]:
lig2 = LayerIntegratedGradients(custom_forward, \
                                [model.longformer.embeddings.word_embeddings, \
                                 model.longformer.embeddings.position_embeddings])

/usr/local/lib/python3.7/dist-packages/captum/attr/_core/layer/layer_integrated_gradients.py:103: UserWarning: Multiple layers provided. Please ensure that each layer is**not** solely solely dependent on the outputs ofanother layer. Please refer to the documentation for moredetail.
  "Multiple layers provided. Please ensure that each layer is"


In [ ]:
attributions2 = lig2.attribute(inputs=(input_ids, position_ids),
                               baselines=(ref_input_ids, ref_position_ids),
                               target=label,
                               n_steps=50,
                               internal_batch_size = 2)

In [ ]:
attributions_word = summarize_attributions(attributions2[0])
attributions_position = summarize_attributions(attributions2[1])

In [ ]:
def get_topk_attributed_tokens(attrs, k=15):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    print(values)
    return top_tokens, values, indices

In [ ]:
def get_botk_attributed_tokens(attrs, k=15):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [ ]:
import pandas as pd
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_word)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_word)


top_pos_start, top_pos_val_start, pos_ind_start = get_topk_attributed_tokens(attributions_position)
bot_pos_start, bot_pos_val_start, pos_ind_start2 = get_botk_attributed_tokens(attributions_position)

df_high = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)],
                   'Position(Index), Attribution': ["{} ({}), {}".format(position, pos, round(val.item(),2)) for position, pos, val in zip(top_pos_start, pos_ind_start, top_pos_val_start)]})

df_low = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(bot_words_start, bot_word_ind_start, bot_words_val_start)],
                   'Position(Index), Attribution': ["{} ({}), {}".format(position, pos, round(val.item(),2)) for position, pos, val in zip(bot_pos_start, pos_ind_start2, bot_pos_val_start)]})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

In [ ]:
df_high

,"Word(Index), Attribution","Position(Index), Attribution"
0,"Ġprogramming (334), 0.24",", (514), 0.32"
1,"Ġprogramming (314), 0.24",", (529), 0.19"
2,", (529), 0.22","ĠAlso (663), 0.12"
3,"Ġprogramming (171), 0.21","X (528), 0.08"
4,"Ġprogramming (84), 0.19","- (1362), 0.05"
5,"Ġsyntax (195), 0.19","Ġsize (117), 0.04"
6,", (514), 0.19",", (664), 0.04"
7,"Ġsyntax (263), 0.17","Ġbe (818), 0.04"
8,"ĠThe (116), 0.17",". (1373), 0.04"
9,"Ġprogramming (445), 0.14","ĠIts (346), 0.04"


In [ ]:
df_low

,"Word(Index), Attribution","Position(Index), Attribution"
0,"ĠTo (151), -0.17","ĠC (1538), -0.72"
1,"ĠThe (468), -0.09","opt (2), -0.37"
2,"utions (1891), -0.09","ĠTo (151), -0.18"
3,"ative (873), -0.08","Ġlanguage (356), -0.05"
4,"ative (822), -0.07",", (154), -0.03"
5,"Ġis (142), -0.06",", (397), -0.03"
6,"Ġtime (642), -0.06","Ġend (153), -0.03"
7,"Ġsolving (148), -0.06","Ġsyntax (195), -0.02"
8,"Ġexpert (231), -0.05","Ġvariable (108), -0.02"
9,"Ġtime (654), -0.05","Ġfocus (1898), -0.02"


In [ ]:
d = {"tokens":all_tokens, "attribution":attributions_word[:len(all_tokens)].cpu()}
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [ ]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens[:15]

,attribution
tokens,
Ġprogramming,1.262697
",",0.847717
Ġsyntax,0.557280
Ġto,0.444532
Ġ,0.366207
Ġthe,0.361825
Ġlanguage,0.352078
Ġand,0.323564
Ġ.,0.302341


In [ ]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution'])
lowest_attrib_tokens[:15]

,attribution
tokens,
.,-0.516404
-,-0.285388
Ġa,-0.267804
Ġof,-0.246206
ative,-0.238762
ĠASP,-0.228577
Ġ(,-0.198362
ĠTo,-0.164541
"),",-0.146611


In [ ]:
from tqdm import tqdm
aggregate_attrib_zero = []
aggregate_attrib_ones = []
aggregate_pos_zero = []
aggregate_pos_ones = []
aggregation_function = {'attribution': 'sum'}

for i in tqdm(range(len(cogs402_ds)), position = 0, leave = True):
  text = cogs402_ds[i]['text']
  label = cogs402_ds[i]['labels']
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  # attention_mask = construct_attention_mask(input_ids)

  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  attributions2 = lig2.attribute(inputs=(input_ids, position_ids),
                               baselines=(ref_input_ids, ref_position_ids),
                               target=label,
                               n_steps=15,
                               internal_batch_size = 2)
  attributions_word = summarize_attributions(attributions2[0])
  attributions_position = summarize_attributions(attributions2[1])

  d = {"tokens":all_tokens, "attribution":attributions_word[:len(all_tokens)].cpu()}  
  d2 = {"tokens":all_tokens, "attribution":attributions_position[:len(all_tokens)].cpu()}  
  
  df_attrib = pd.DataFrame(d)
  df_attrib2 = pd.DataFrame(d2)
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_function)
  df_attrib2 = df_attrib2.groupby(df_attrib2['tokens']).aggregate(aggregation_function)

  if label == 0:
    aggregate_attrib_zero.append(df_attrib)
    aggregate_pos_zero.append(df_attrib2)
  else:
    aggregate_attrib_ones.append(df_attrib)
    aggregate_pos_ones.append(df_attrib2)

100%|██████████| 1179/1179 [3:15:46<00:00,  9.96s/it]


In [ ]:
def combinedataframe(listframes, aggregation_func):
  df_attrib = pd.concat(listframes)
  df_attrib = df_attrib.reset_index(level=0)
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_func)
  df_attrib['attribution'] = df_attrib['attribution'].div(len(listframes))
  highest_attrib_tokens_all = df_attrib.sort_values(by=['attribution'], ascending=False)
  return highest_attrib_tokens_all

In [ ]:
df_attrib_zero = combinedataframe(aggregate_attrib_zero, aggregation_function)
df_attrib_ones = combinedataframe(aggregate_attrib_ones, aggregation_function)
df_pos_zero = combinedataframe(aggregate_pos_zero, aggregation_function)
df_pos_ones = combinedataframe(aggregate_pos_ones, aggregation_function)

In [ ]:
df_attrib_zero[:10]

,attribution
tokens,
Ġ,0.412459
Ġprogramming,0.353186
Ġ.,0.305509
Ġand,0.222657
Ġlanguages,0.180339
",",0.177019
Ġlanguage,0.122011
ĠJava,0.112718
Ġcode,0.110560


In [ ]:
df_attrib_ones[:10]

,attribution
tokens,
.,0.564723
-,0.216672
Ġlearning,0.214803
Ġin,0.154177
Ġneural,0.131543
Ġa,0.127753
Ġtraining,0.092751
Ġ(,0.088613
Ġon,0.077409


In [ ]:
df_pos_zero[:10]

,attribution
tokens,
Ġthe,0.467113
Ġa,0.330406
",",0.298435
.,0.242746
Ġand,0.162589
Ġof,0.160188
Ġis,0.118130
Ġto,0.091815
Ġfor,0.074328


In [ ]:
df_pos_ones[:10]

,attribution
tokens,
Ġ,0.294896
-,0.062223
Ġ(,0.051008
Ġ.,0.036911
Ġlanguage,0.031438
Ġneural,0.030427
X,0.030159
Ġ[,0.024737
),0.023960


In [ ]:
df_attrib_zero.to_csv('data/word_emb_attrib_zero_papers.csv')  
df_attrib_ones.to_csv('data/word_emb_attrib_ones_papers.csv')  
df_pos_zero.to_csv('data/pos_emb_attrib_zero_papers.csv')  
df_pos_ones.to_csv('data/pos_emb_attrib_ones_papers.csv')  